In [1]:
import sys
sys.path.insert(0,'C:\\code\\python_for_the_financial_economist\\')

"""
Magic commands
"""

%load_ext autoreload
%autoreload 2

# animations, etc. requires below magic command
# %matplotlib notebook


"""
Load relevant packages
"""

import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# matrix square root
from scipy.linalg import sqrtm
from scipy import stats

# packages for convex programming
import cvxopt
import cvxpy as cp


"""
Own packages
"""



'\nOwn packages\n'

# Convex optimization

Convex optimization relates to optimization problems where the objective function is a convex function subject to only convex and affine constraints. 

The main advantage of convex programming problems is that there is only on local minimum, which is also globally optimal, and that we have can use efficient algorithms to solve such problems numerically. 

In finance, we will encounter convex optimization problems, in particular linear and quadratic programming problems, in many application, e.g. portfolio optimization. 

This Jupyter notebook will give a brief introduction to convex programming problems relevant for primarily portfolio optimization while the interested reader is refered to [Boyd and Vandenberghe (2004)](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf). The focus is on how to implement convex optimization problems using the `cvxopt` and `cvxpy` packages. 

## The general problem 

Convex programming solves the problem 

$$
\begin{align*}
	\min_{\mathbf{x}}  \quad & f(\mathbf{x}) \\
	\textrm{s.t.} \quad & \mathbf{A} \mathbf{x} = \mathbf{b}\\
	 & g(\mathbf{x}) \leq \mathbf{0}
\end{align*}
$$

where $\mathbf{x}$ is a n-dimensional vector, $f : \mathbb{R}^n \to \mathbb{R}$, $\mathbf{A}$ and $\mathbf{b}$ defines an affine set, $g(\mathbf{x}) = (g_1 (\mathbf{x}), ..., g_K (\mathbf{x}) )^\top$ is a vector of convex functions defining a convex set. 

## Linear programming

In linear programming the convex objective function is required to be linear and we only allow for linear equality and inequality constraints. 

$$
\begin{align*}
	\min_{\mathbf{x}}  \quad & \mathbf{c}^\top \mathbf{x} \\
	\textrm{s.t.} \quad & \mathbf{A} \mathbf{x} = \mathbf{b}\\
	 & \mathbf{G} \mathbf{x} \leq \mathbf{h}
\end{align*}
$$

where $\mathbf{A}$ is a $m \times n$ matrix, $\mathbf{b}$ is a $m$-dimensional vector, where $m$ is the number of equality constraints.

__Example: Solving a linear program__

Note: Example is taken from [CVXOPT tutorial](https://cvxopt.org/examples/tutorial/lp.html). 

Consider the linear program 

$$
\begin{align*}
	\arg \min_{\mathbf{x}}  \quad & 2 x_1 + x_2\\
	\textrm{s.t.} \quad & -x_1 + x_2 \leq 1 \\
	 & x_1 + x_2 \geq 2 \\
     & x_2 \geq 0 \\
     & x_1 - 2x_2 \leq 4
\end{align*}
$$


Solve the problem. 

In [2]:
"""
CVXOPT
"""

# define input matrices
c = cvxopt.matrix([2.0, 1.0])
G = cvxopt.matrix([[-1.0, -1.0, 0.0, 1.0],[1.0, -1.0, -1.0, -2.0]])
h = cvxopt.matrix([ 1.0, -2.0, 0.0, 4.0 ])

# solve problem 
sol = cvxopt.solvers.lp(c, G, h)

     pcost       dcost       gap    pres   dres   k/t
 0:  2.6471e+00 -7.0588e-01  2e+01  8e-01  2e+00  1e+00
 1:  3.0726e+00  2.8437e+00  1e+00  1e-01  2e-01  3e-01
 2:  2.4891e+00  2.4808e+00  1e-01  1e-02  2e-02  5e-02
 3:  2.4999e+00  2.4998e+00  1e-03  1e-04  2e-04  5e-04
 4:  2.5000e+00  2.5000e+00  1e-05  1e-06  2e-06  5e-06
 5:  2.5000e+00  2.5000e+00  1e-07  1e-08  2e-08  5e-08
Optimal solution found.


In [3]:
np.array(sol['x'])

array([[0.5],
       [1.5]])

In [4]:
"""
CVXPY
"""

# define variables
c = np.array([2.0, 1.0])
G = np.array([[-1.0, -1.0, 0.0, 1.0],[1.0, -1.0, -1.0, -2.0]]).T
h = np.array([ 1.0, -2.0, 0.0, 4.0 ])

# define problem 
x = cp.Variable(2)
prob = cp.Problem(cp.Minimize(c @ x), constraints=[G @ x <= h])

# solve problem
prob.solve()

# solution 
x.value

array([0.5, 1.5])

## Quadratic programming

We will often encounter quadratic programming problems in finance and economics, e.g. in relation to portfolio optimization and linear regression. Quadratic programming solves the optimization problem 

$$
\begin{align*}
	\min_{\mathbf{x}}  \quad & \frac{1}{2} \mathbf{x}^\top  \mathbf{P}  \mathbf{x}  +  \mathbf{q}^\top \mathbf{x} \\
	\textrm{s.t.} \quad & \mathbf{A} \mathbf{x} = \mathbf{b}\\
	 & \mathbf{G} \mathbf{x} \leq \mathbf{h}
\end{align*}
$$

Clearly, linear programming is a special case of quadratic programming. 


__Example:  Portofolio optimization - quadratic utility__


Assume that an investor has a quadratic utility function 

$$
U(\mathbf{w}) = \mathbf{w}^\top \boldsymbol{\mu} - \frac{\lambda}{2} \mathbf{w}^\top \boldsymbol{\Sigma} \mathbf{w}
$$

where $\mathbf{w}$ is the portfolio weights, $\boldsymbol{\mu}$ is a vector of expected returns, and $\boldsymbol{\Sigma}$ denotes the covariance matrix. 

The investor seeks to maximize the utility given the constraint that the portfolio weights sum to one

$$
\mathbf{1}^\top \mathbf{w} = 1
$$

The solution to this optimization problem is given by (see e.g. [Rebonato and Denev, "Portfolio Management under stress"](https://www.amazon.com/Portfolio-Management-under-Stress-Bayesian-Net/dp/1107048117))

$$
\mathbf{w}^* = \frac{1}{\lambda} \boldsymbol{\Sigma}^{-1} \boldsymbol{\mu} - \frac{1}{\lambda} \boldsymbol{\Sigma}^{-1} \mathbf{A}^\top \mathbf{C}^{-1} \left(\mathbf{A}\boldsymbol{\Sigma}^{-1}\boldsymbol{\mu}  - \lambda \mathbf{b} \right)
$$

with $\mathbf{A} = \mathbf{1}^\top$, $\mathbf{b} = 1$, and $\mathbf{C} = \mathbf{A} \boldsymbol{\Sigma}^{-1}\mathbf{A}^\top$. Note that this solution is valid for more general constraints of the form $\mathbf{A}^\top \mathbf{w} = \mathbf{b}$. 


We assume that 

$$
\boldsymbol{\mu}  = \begin{bmatrix} 0.02 \\ 0.04 \\ 0.08 \end{bmatrix}, \; \mathbf{v} = \begin{bmatrix} 0.075 \\ 0.15 \\ 0.3 \end{bmatrix}, \; \textbf{Corr} = \begin{bmatrix} 1.0 & 0.2 & 0.1 \\
                     0.2 & 1.0 & 0.4 \\
                     0.1 & 0.4 & 1.0 \end{bmatrix}
$$

and $\lambda = 5$. 

Compare the solution to the one obtained using `cvxopt` or `cvxpy`.

In [5]:
"""
Define input parameters
"""

mu = np.array([0.02, 0.04, 0.08])

vols = np.array([0.075, 0.15, 0.3])

corr_mat = np.array([[1.0, 0.2, 0.1],
                     [0.2, 1.0, 0.4],
                     [0.1, 0.4, 1.0]])

cov_mat = np.outer(vols, vols) * corr_mat
cov_mat

array([[0.005625, 0.00225 , 0.00225 ],
       [0.00225 , 0.0225  , 0.018   ],
       [0.00225 , 0.018   , 0.09    ]])

In [6]:
def calculate_optimal_weights(mu: np.ndarray, cov_matrix: np.ndarray, risk_aversion: float) -> np.ndarray: 
    
    """
    Function that calculates optimal port. weights
    
    Parameters
    ----------
    mu: 
        Expected returns
    cov_matrix: 
        Covariance matrix
    risk_aversion: 
        Risk aversion parameter
    
    Returns
    -------
    float
        Optimal portfolio weights
    """
    sigma_inv = np.linalg.inv(cov_matrix)
    A = np.ones_like(mu)
    C = A @ sigma_inv @ A
    C_inv = 1.0 / C
    b = 1.0
    
    first_part = 1.0 / risk_aversion * sigma_inv @ mu 
    second_part = 1.0 / risk_aversion * sigma_inv @ A * C_inv * (A @ sigma_inv @ mu - risk_aversion * b)
    
    opt_weights = first_part - second_part
    
    return opt_weights

In [7]:
risk_aversion = 5
print("optimal weights: ")
w_opt = calculate_optimal_weights(mu, cov_mat, risk_aversion)
w_opt

optimal weights: 


array([0.66414857, 0.2115203 , 0.12433113])

In [8]:
"""
CVXOPT
"""

P = cvxopt.matrix(cov_mat * risk_aversion) #<- we need to minimize
q = cvxopt.matrix(-mu) #<- we need to minimize
A = cvxopt.matrix([[1.0, 1.0, 1.0]]).T
b = cvxopt.matrix([1.0])

# solve problem 
sol = cvxopt.solvers.qp(P, q, A=A, b=b)

# solution 
np.array(sol['x']) # <- gives the same solution as the analytical formula

array([[0.66414857],
       [0.2115203 ],
       [0.12433113]])

In [9]:
"""
CVXPY
"""
num_assets = len(mu)
# define variables
P = cov_mat * risk_aversion
q = mu
A = np.ones(num_assets)
b = 1.0


# define problem 
x = cp.Variable(num_assets)
prob = cp.Problem(cp.Maximize(q @ x - 0.5 * cp.quad_form(x, P)), constraints=[cp.sum(x) == 1]) # cp.sum(x) == 1 or  A @ x == b

# solve problem
prob.solve()

# solution 
x.value

array([0.66414857, 0.2115203 , 0.12433113])

If add the constraint that all assets need an allocation above 30%, then we do not have an analytical solution anymore - but luckily we can apply `cvxopt` or `cvxpy`. 

In [10]:
num_assets = len(mu)
# define variables
P = cov_mat * risk_aversion
q = mu
A = np.ones(num_assets)
b = 1.0


# define problem 
x = cp.Variable(num_assets)
prob = cp.Problem(cp.Maximize(q @ x - 0.5 * cp.quad_form(x, P)),
                  constraints=[A @ x == b,
                               x >= 0.3]) # <- just add this input, cvxpy will set the matrices up correctly!

# solve problem
prob.solve()

# solution 
x.value

array([0.4, 0.3, 0.3])

## Second-order cone programming

A second-order cone program is an optimization problem of the form (following the notation from [CVXPY](https://www.cvxpy.org/examples/basic/socp.html))

$$
\begin{align*}
	\arg \min_{\mathbf{x}}  \quad & \mathbf{f}^\top \mathbf{x} \\
	\textrm{s.t.} \quad & \Vert \mathbf{A}_i \mathbf{x} + \mathbf{b}_i \Vert_2  \leq \mathbf{c}_i^\top \mathbf{x} + d_i, \; i = 1, ..., m\\
	 & \mathbf{F} \mathbf{x} = \mathbf{g}
\end{align*}
$$

where $\mathbf{x}$ is the optimization variable,  $\mathbf{f}$ is a $n$-dimensional vector, $\mathbf{A}_i$ is a $n_i \times n$  matrix, $\mathbf{b}_i$ is a $n_i$-dimensional vector, $\mathbf{c}_i$ is a $n$-dimensional vector, $d_i$ is a scalar, $\mathbf{F}$ is a $p \times n$ matrix and $\mathbf{g}$ is a $p$-dimensional vector. 

__Example: Portfolio optimization with VaR constraint__

Assume that the net return is multivariate normal $\mathbf{R} \sim \text{MN}(\boldsymbol{\mu}, \boldsymbol{\Sigma})$ with

$$
\boldsymbol{\mu}  = \begin{bmatrix} 0.02 \\ 0.04 \\ 0.08 \end{bmatrix}, \; \mathbf{v} = \begin{bmatrix} 0.075 \\ 0.15 \\ 0.3 \end{bmatrix}, \; \textbf{Corr} = \begin{bmatrix} 1.0 & 0.2 & 0.1 \\
                     0.2 & 1.0 & 0.4 \\
                     0.1 & 0.4 & 1.0 \end{bmatrix}
$$

Assume that we want to maximize the expected return subject to the (negative) Value at Risk  being larger certain threshold, $b$, 

$$
\begin{align*}
	\max_{\mathbf{w}}  \quad & \boldsymbol{\mu}^\top \mathbf{w} \\
	\textrm{s.t.} \quad & \boldsymbol{\mu}^\top \mathbf{w} + \Phi^{-1}(\alpha) \sqrt{\mathbf{w}^\top \boldsymbol{\Sigma} \mathbf{w}} \geq b\\
     & \mathbf{w} \geq \mathbf{0} \\
	 & \mathbf{1}^\top \mathbf{w} = 1
\end{align*}
$$

Since $\boldsymbol{\mu}^\top \mathbf{w} + \Phi^{-1}(\alpha) \sqrt{\mathbf{w}^\top \boldsymbol{\Sigma} \mathbf{w}} \geq b$ is equivalent to $\boldsymbol{\mu}^\top \mathbf{w} + \Phi^{-1}(\alpha) \Vert \boldsymbol{\Sigma}^{1/2} \mathbf{w} \Vert \geq b$, we can equivalently write the problem as 

$$
\begin{align*}
	\max_{\mathbf{w}}  \quad & \boldsymbol{\mu}^\top \mathbf{w} \\
	\textrm{s.t.} \quad & \Vert \boldsymbol{\Sigma}^{1/2} \mathbf{w} \Vert \leq \frac{1}{\Phi^{-1}(\alpha)}\boldsymbol{\mu}^\top \mathbf{w} -\frac{1}{\Phi^{-1}(\alpha)}b\\
     & \mathbf{w} \geq \mathbf{0} \\    
	 & \mathbf{1}^\top \mathbf{w}= 1
\end{align*}
$$

which is a second-order cone programming problem. 

In [11]:
"""
Define input parameters
"""

mu = np.array([0.02, 0.04, 0.08])

vols = np.array([0.075, 0.15, 0.3])

corr_mat = np.array([[1.0, 0.2, 0.1],
                     [0.2, 1.0, 0.4],
                     [0.1, 0.4, 1.0]])

cov_mat = np.outer(vols, vols) * corr_mat

In [12]:
# define variables, etc. 
tail = 0.05
loss = 0.14112535684744942 # minimum attainable VaR(5%) is ~14.1%

f = mu
A = sqrtm(cov_mat) 
c = mu / stats.norm.ppf(tail)
d = -loss /  stats.norm.ppf(tail)

# variables 
w = cp.Variable(num_assets)

# constraints 
constraints = [cp.sum(w) == 1, w >= 0]
constraints += [cp.SOC(c @ w + d, A @ w)]

# problem 
prob = cp.Problem(cp.Maximize(f @ w), constraints=constraints)

# solve problem
prob.solve(verbose=False)

# solution 
w.value

array([0.85550138, 0.13601139, 0.00848723])

## Semidefinite programming

A semidefinite program (SDP) is an optimization problem of the form 

$$
\begin{align*}
	\min_{\mathbf{X}}  \quad &  \text{tr}(\mathbf{C} \mathbf{X}) \\
	\textrm{s.t.} \quad & \text{tr}(\mathbf{A}_i \mathbf{X}) = b_i, \; i = 1, ..., m\\
    & \mathbf{X} \succeq 0
\end{align*}
$$

where $\text{tr}$ is the [trace operator](https://en.wikipedia.org/wiki/Trace_(linear_algebra)). The optimization variable $\mathbf{X} \in \mathcal{S}^n$ is a $n \times n$ symmetric matrix. The problem data $\mathbf{C}, \mathbf{A}_1, ..., \mathbf{A}_m \in \mathcal{S}^n$ are $n \times n$ symmetric matrices. $\mathbf{X} \succeq 0$ is a matrix inequality requiring $\mathbf{X}$ to be a symmetric and positive semidefinite matrix ($\succ$ means positive definite matrix). 

See e.g. [Boyd and Vandenberghe (2004)](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf) and [Freund (2004)](https://ocw.mit.edu/courses/15-084j-nonlinear-programming-spring-2004/a632b565602fd2eb3be574c537eea095_lec23_semidef_opt.pdf) for additional information.

__Example: How to set up a problem__

Note: Example taken from [Freund (2004)](https://ocw.mit.edu/courses/15-084j-nonlinear-programming-spring-2004/a632b565602fd2eb3be574c537eea095_lec23_semidef_opt.pdf). 

Consider the case with $m=2$ and $n = 3$ with the following matrices 

$$
\mathbf{A}_1 = \begin{pmatrix} 1 & 0 & 1 \\ 0 & 3 & 7 \\ 1 & 7 & 5 \end{pmatrix}, \quad
\mathbf{A}_2 = \begin{pmatrix} 0 & 2 & 8 \\ 2 & 6 & 0 \\ 8 & 0 & 4 \end{pmatrix}, \quad
\mathbf{C} = \begin{pmatrix} 1 & 2 & 3 \\ 2 & 9 & 0 \\ 3 & 0 & 7 \end{pmatrix}
$$

and the values $b_1 = 11$ and $b_2 = 19$. The optimization variable $\mathbf{X}$ will be a $3 \times 3$ symmetric matrix: 

$$
\mathbf{X} = \begin{pmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{pmatrix}
$$

Doing some tedios matrix calculations, we are able to show that the problem is given by (using that $\mathbf{X}$ is symmetric, e.g. $x_{12} = x_{21}$)

$$
\begin{align*}
	\min  \quad &  x_{11} + 4 x_{12} + 6 x_{13} + 9 x_{22} + 0 x_{23} + 7 x_{33} \\
	\textrm{s.t.} \quad & x_{11} + 0 x_{12} + 2 x_{13} + 3 x_{22} + 14 x_{23} + 5 x_{33} = 11, \\
    & 0 x_{11} + 4 x_{12} + 16 x_{13} + 6 x_{22} + 0 x_{23} + 4 x_{33} = 19 \\
    & \begin{pmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{pmatrix} \succeq 0
\end{align*}
$$

In the following code-block, we show how to implement the problem with `cvxpy`

In [13]:
"""
Define relevant matrices
"""


A1 = np.array([[1, 0, 1], 
               [0, 3, 7], 
               [1, 7, 5]])

b1 = 11

A2 = np.array([[0, 2, 8], 
               [2, 6, 0], 
               [8, 0, 4]])

b2 = 19

C = np.array([[1, 2, 3], 
              [2, 9, 0], 
              [3, 0, 7]])

"""
Define X matrix - optimization variable
"""

X = cp.Variable((3, 3), symmetric=True) # tell cvxpy that X is a symmetric n x n matrix

"""
Define constraints
"""

constraints = [X >> 0] # matrix X is positive definite

constraints.append(cp.trace(A1 @ X) == b1)
constraints.append(cp.trace(A2 @ X) == b2)

"""
Define problem 
"""

prob = cp.Problem(cp.Minimize(cp.trace(C @ X)), constraints)

"""
Solve problem and print solution
"""

prob.solve()

print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 13.902182526039745
A solution X is
[[1.05600838 0.36917833 0.86830385]
 [0.36917833 0.12907089 0.30358367]
 [0.86830385 0.30358367 0.71399996]]


__Example: Relation to linear programming__

If we had a linear programming problem of the form 

$$
\begin{align*}
	\min_{\mathbf{x}}  \quad & \mathbf{c}^\top \mathbf{x} \\
	\textrm{s.t.} \quad & \mathbf{A} \mathbf{x} = \mathbf{b}\\
\end{align*}
$$

We would be able to define an equivalent semidefinite programming problem by defining 

$$
\mathbf{A}_i = \begin{pmatrix} a_{i1} & 0 &  \dots & 0 \\ 0 & a_{i2} & \dots & 0 \\ \vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & a_{in}\end{pmatrix}, i = 1, \dots, m \quad \text{and} \quad  \mathbf{C} = \begin{pmatrix} c_1 & 0 &  \dots & 0 \\ 0 & c_2 & \dots & 0 \\ \vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & c_n\end{pmatrix}
$$

such that we can write the linear programming problem as 

$$
\begin{align*}
	\min_{\mathbf{X}}  \quad &  \text{tr}(\mathbf{C} \mathbf{X}) \\
	\textrm{s.t.} \quad & \text{tr}(\mathbf{A}_i \mathbf{X}) = b_i, \; i = 1, ..., m\\
    & x_{ij} = 0, i = 1, \dots, n, j = i + 1, \dots, n \\
    & \mathbf{X} \succeq 0
\end{align*}
$$

Thus, we are restricting $\mathbf{X}$ to be a diagonal matrix. 

In practice, we would not transform a linear programming problem to a semidefinite one, but this illustrates that a semidefinite programming problem includes linear programming as a special case. 

__Example: Maximization of entry in correlation matrix__

Note: This example is adapted from [[W]](https://en.wikipedia.org/wiki/Semidefinite_programming). 

Let $\mathbf{X}$ be a $3 \times 3$ correlation matrix 


$$
\mathbf{X} = \begin{pmatrix} 1 & x_{12} & x_{13} \\ x_{21} & 1 & x_{23} \\ x_{31} & x_{32} & 1 \end{pmatrix} \succeq 0
$$

We assume that $x_{12} = -0.15$ and $x_{23} = 0.45$. What is the maximum value that $x_{13}$ can take?

In [14]:
n = 3

"""
Define X matrix - optimization variable
"""

X = cp.Variable((n, n), symmetric=True) # tell cvxpy that X is a symmetric n x n matrix

"""
Define constraint matrices
"""

# positive definite constraint
constraints = [X >> 0]

# add diagonal constraints - diagonal must be equal to one
for i in range(n):
    A = np.zeros((n,n))
    A[i, i] = 1.0
    
    constraints.append(cp.trace(A @ X) == 1.0)
    
# add constraints on x12 and x23 
A = np.zeros((n,n))
A[0, 1] = 1.0

constraints.append(cp.trace(A @ X) == -0.15)

A = np.zeros((n,n))
A[1, 2] = 1.0

constraints.append(cp.trace(A @ X) == 0.45)


"""
Define problem 
"""

C = np.zeros((n, n))
C[0, 2] = C[2, 0] = 0.5

prob = cp.Problem(cp.Maximize(cp.trace(C @ X)), constraints)

"""
Solve problem and print solution
"""

prob.solve()

print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 0.8154248299028961
A solution X is
[[ 1.         -0.15        0.81542483]
 [-0.15        1.          0.45      ]
 [ 0.81542483  0.45        1.        ]]


Instead of specifying specific values for $x_{12}$ and $x_{23}$, we could assume $-0.2 \leq x_{12} \leq -0.1$ and $0.4 \leq x_{12} \leq 0.5$. 

To add inequality constraints, we can add slack variables to the problem, e.g. $x_{12} \leq -0.1$ can be implemented as 

$$
\text{tr} \left(\left(\begin{array}{cccccc}
0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\end{array}\right)\cdot\left(\begin{array}{cccccc}
1 & x_{12} & x_{13} & 0 & 0 & 0\\
x_{12} & 1 & x_{23} & 0 & 0 & 0\\
x_{13} & x_{23} & 1 & 0 & 0 & 0\\
0 & 0 & 0 & s_{1} & 0 & 0\\
0 & 0 & 0 & 0 & s_{2} & 0\\
0 & 0 & 0 & 0 & 0 & s_{3}\end{array}\right)\right)=x_{12} + s_{1}=-0.1
$$

Since $s_1$ is restricted to be positive, then $x_{12}$ have to be smaller than $-0.1$. 

In [15]:
n = 3
num_ineq = 4
n_tot = n + num_ineq

"""
Define X matrix - optimization variable
"""

X = cp.Variable((n_tot, n_tot), symmetric=True) # tell cvxpy that X is a symmetric n x n matrix

"""
Define constraint matrices
"""

# positive definite constraint
constraints = [X >> 0]

# add diagonal constraints - diagonal must be equal to one
for i in range(n):
    A = np.zeros((n_tot, n_tot))
    A[i, i] = 1.0
    
    constraints.append(cp.trace(A @ X) == 1.0)
    
# add constraints on x12
A = np.zeros((n_tot, n_tot))
A[0, 1] = 1.0
A[3, 3] = 1.0

constraints.append(cp.trace(A @ X) == -0.1)

A = np.zeros((n_tot, n_tot))
A[0, 1] = 1.0
A[4, 4] = -1.0

constraints.append(cp.trace(A @ X) == -0.2)

# add constraints on x23
A = np.zeros((n_tot, n_tot))
A[1, 2] = 1.0
A[5, 5] = 1.0

constraints.append(cp.trace(A @ X) == 0.5)

A = np.zeros((n_tot, n_tot))
A[1, 2] = 1.0
A[6, 6] = -1.0

constraints.append(cp.trace(A @ X) == 0.4)


"""
Define problem 
"""

C = np.zeros((n_tot, n_tot))
C[0, 2] = C[2, 0] = 0.5

prob = cp.Problem(cp.Maximize(cp.trace(C @ X)), constraints)

"""
Solve problem and print solution
"""

prob.solve(verbose=False)

print("The optimal value is", prob.value)
print("A solution X is")
print(X.value[:3, :3])

The optimal value is 0.8719398868277205
A solution X is
[[ 1.         -0.09998599  0.87193989]
 [-0.09998599  1.00000001  0.39998524]
 [ 0.87193989  0.39998524  1.        ]]


# References

## Lecture notes

[Freund (2004)](https://ocw.mit.edu/courses/15-084j-nonlinear-programming-spring-2004/a632b565602fd2eb3be574c537eea095_lec23_semidef_opt.pdf)

## Books

[Convex Optimization, Boyd and Vandenberghe (2004)](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)